In [1]:
import json

In [2]:
with open(r'C:\workspace\misc\5980\aokvqa\aokvqa_v1p0_val.json', 'rb') as fp:
    data = json.load(fp)

In [5]:
questions = [item['question'] for item in data]

In [21]:
choices = [item['choices'] for item in data]

In [22]:
image_ids = [item['image_id'] for item in data]

In [23]:
len(questions), len(choices), len(image_ids)

(1145, 1145, 1145)

In [14]:
questions[1000:]

['How many animal trunks are visible here?',
 'What U.S. city is the most likely setting for this street?',
 'What is the man doing with the ropes in his hands?',
 "What is holding the man's feet?",
 'What could keep this food from falling apart?',
 'What is the most diverse seashore bird?',
 'What service is offered for free behind the woman in red?',
 'What kind of gathering does this look like?',
 'What has this stove been used for?',
 'What type of telephone is in the room?',
 'What does the blue sign indicate?',
 'Which person is controlling the horse?',
 'What ingredient is used first on the pizza?',
 'What century were these invented in?',
 'What type of shop is to the right of the silver bus?',
 'What kind of landscapes are present?',
 'In what environment is the meal taking place?',
 'Athletes often participate in what activity to warm up their muscles before starting?',
 'What are the zebras doing?',
 'What is powering the bikes?',
 'What animal has a similar nose to this ani

In [8]:
import re
import spacy
nlp = spacy.load("en_core_web_sm")

def extract_subject(question: str):
    """Roughly extract the noun phrase or main object of the question."""
    doc = nlp(question)
    # Prefer object of preposition or direct object
    for tok in doc:
        if tok.dep_ in ("pobj", "dobj", "nsubj", "attr"):
            span = " ".join([t.text for t in tok.subtree])
            if len(span.split()) > 1:
                return span
    # fallback to last noun chunk
    noun_chunks = list(doc.noun_chunks)
    if noun_chunks:
        return noun_chunks[-1].text
    return question

def question_to_statements(question, choices):
    q = question.lower().strip().rstrip("?")
    sents = []
    
    subj = extract_subject(question)
    
    # 1️⃣ what is / what are
    if re.search(r"\bwhat (is|are)\b", q):
        if "in " in q:
            container = q.split("in",1)[-1].strip()
            for c in choices:
                sents.append(f"{container} has {c} in it")
        elif "on " in q:
            container = q.split("on",1)[-1].strip()
            for c in choices:
                sents.append(f"{container} has {c} on it")
        elif "made of" in q or "made from" in q:
            target = q.split("made")[0].replace("what is","").strip()
            for c in choices:
                sents.append(f"{target} is made of {c}")
        elif "used for" in q:
            target = q.split("used")[0].replace("what is","").strip()
            for c in choices:
                sents.append(f"{target} is used for {c}")
        else:
            for c in choices:
                sents.append(f"{subj} is {c}")
        return sents
    
    # 2️⃣ what kind/type
    if "what kind" in q or "what type" in q:
        for c in choices:
            sents.append(f"{subj} is a type of {c}")
        return sents

    # 3️⃣ where / location questions
    if q.startswith("where") or "where" in q:
        for c in choices:
            sents.append(f"{subj} is located in {c}")
        return sents
    
    # 4️⃣ who questions
    if q.startswith("who") or "who" in q:
        for c in choices:
            sents.append(f"{subj} is {c}")
        return sents

    # 5️⃣ why / purpose questions
    if q.startswith("why") or "why" in q:
        for c in choices:
            sents.append(f"{subj} is because of {c}")
        return sents
    
    # 6️⃣ which number / how many
    if "which number" in q or "how many" in q:
        for c in choices:
            sents.append(f"The number is {c}")
        return sents

    # 7️⃣ what color / appearance
    if "what color" in q or "what colour" in q or "what pattern" in q:
        for c in choices:
            sents.append(f"{subj} is {c} in color")
        return sents

    # 8️⃣ what country / nationality
    if "what country" in q:
        for c in choices:
            sents.append(f"{subj} belongs to {c}")
        return sents

    # 9️⃣ what is the person doing
    if re.search(r"what.*doing", q):
        for c in choices:
            sents.append(f"{subj} is doing {c}")
        return sents

    # 🔟 fallback: generic declarative pattern
    for c in choices:
        sents.append(f"{subj} is {c}")
    return sents


In [42]:
import re

def remove_wh_questions(text):
    """
    Removes WH-question words (who, what, when, where, why, how, etc.) from the input text.
    """
    # Define common WH-question keywords
    wh_words = [
        r'\bwho\b', r'\bwhom\b', r'\bwhose\b',
        r'\bwhat\b', r'\bwhich\b', r'\bwhen\b',
        r'\bwhere\b', r'\bwhy\b', r'\bhow\b'
    ]
    
    # Create a regex pattern that matches any WH-word, case-insensitive
    pattern = re.compile('|'.join(wh_words), re.IGNORECASE)
    
    # Replace matched words with empty string
    cleaned_text = pattern.sub('', text)
    
    # Clean up extra spaces
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
    
    return cleaned_text


In [25]:
import re
import spacy
nlp = spacy.load("en_core_web_sm")

# --------------------------
# Helper functions
# --------------------------

def add_article(word: str):
    """Add 'a' or 'an' before a word when appropriate."""
    if not word:
        return word
    word = word.strip()
    if word[0].lower() in "aeiou":
        return f"an {word}"
    return f"a {word}"

def capitalize(s: str):
    """Capitalize first letter and ensure ending period."""
    s = s.strip()
    if not s:
        return s
    s = s[0].upper() + s[1:]
    if not s.endswith('.'):
        s += '.'
    return s


# --------------------------
# Core conversion
# --------------------------

def convert_question_to_statements(question: str, choices: list[str]) -> list[str]:
    """Convert question + choices to declarative statements (pure NLP)."""
    q = question.strip()
    q_lower = q.lower().strip("? ")
    results = []

    # ------------- regex-based templates ---------------

    def templ(subj, template):
        for c in choices:
            sent = template.format(subj=subj, choice=c)
            results.append(capitalize(sent))
        return results

    # What is / are ...?
    m = re.match(r"^what (is|are) (.*)", q_lower)
    if m:
        subj = m.group(2)
        return templ(subj, "The {subj} is {choice}")

    # What type/kind/sort of ...?
    m = re.match(r"^what (type|kind|sort) of (.*)", q_lower)
    if m:
        subj = m.group(2)
        return templ(subj, "The {subj} is {choice}")

    # What does ... mean / indicate / say?
    m = re.match(r"^what does (.*) (mean|indicate|say|represent)", q_lower)
    if m:
        subj = m.group(1)
        verb = m.group(2)
        return templ(subj, f"The {{subj}} {verb}s {{choice}}")

    # What is ... used for?
    m = re.match(r"^what is (.*) used for", q_lower)
    if m:
        subj = m.group(1)
        return templ(subj, "The {subj} is used for {choice}")

    # Why is ... ?
    m = re.match(r"^why is (.*)", q_lower)
    if m:
        subj = m.group(1)
        return templ(subj, "The {subj} because {choice}")

    # Where is ... ?
    m = re.match(r"^where is (.*)", q_lower)
    if m:
        subj = m.group(1)
        return templ(subj, "The {subj} is in {choice}")

    # How many ... ?
    m = re.match(r"^how many (.*)", q_lower)
    if m:
        subj = m.group(1)
        return templ(subj, "There are {choice} {subj}")

    # Which ... ?
    m = re.match(r"^which (.*)", q_lower)
    if m:
        subj = m.group(1)
        return templ(subj, "The {subj} is {choice}")

    # ... is called what?
    m = re.match(r"^(.*) is called what", q_lower)
    if m:
        subj = m.group(1)
        return templ(subj, "The {subj} is called {choice}")

    # ... has what ...
    m = re.match(r"^(.*) has what (.*)", q_lower)
    if m:
        subj, rest = m.groups()
        return templ(subj, "The {subj} has {choice} " + rest)

    # ... used for what?
    m = re.match(r"^(.*) used for what", q_lower)
    if m:
        subj = m.group(1)
        return templ(subj, "The {subj} is used for {choice}")

    # In what ... ?
    m = re.match(r"^in what (.*)", q_lower)
    if m:
        subj = m.group(1)
        return templ(subj, "In {choice} {subj}")

    # How ... ?
    m = re.match(r"^how (.*)", q_lower)
    if m:
        subj = m.group(1)
        return templ(subj, "The {subj} {choice}")

    # ---------------- spaCy fallback ----------------

    doc = nlp(q.strip("?"))
    root = [t for t in doc if t.head == t][0]
    subj = " ".join([t.text for t in doc if t.dep_ in ("nsubj", "nsubjpass")])
    pobj = " ".join([t.text for t in doc if t.dep_ in ("pobj", "dobj", "attr")])

    for c in choices:
        if subj:
            sent = f"{subj} {root.lemma_} {c} {pobj}".strip()
        else:
            sent = f"{q.strip('?')} — {c}"
        results.append(capitalize(sent))

    return results


In [32]:
question_to_statements(questions[1], choices[1])

['The number is one',
 'The number is ten',
 'The number is nine',
 'The number is thirty']

In [26]:
convert_question_to_statements(questions[0], choices[0])

["The in the motorcyclist's mouth is toothpick.",
 "The in the motorcyclist's mouth is food.",
 "The in the motorcyclist's mouth is popsicle stick.",
 "The in the motorcyclist's mouth is cigarette."]

In [45]:
result = question_to_statements(questions[0], choices[0])
result = [remove_wh_questions(s) for s in result]
result

["the motorcyclist's mouth has toothpick in it",
 "the motorcyclist's mouth has food in it",
 "the motorcyclist's mouth has popsicle stick in it",
 "the motorcyclist's mouth has cigarette in it"]

In [34]:
import random
import re

def make_contextual_choices(question, choices):
    """
    Creates 4 contextual answer choices that fit naturally with any type of question.
    Works for:
      - "Which of the following is a programming language?"
      - "Which animal can fly?"
      - "What best describes the pool of water?"
      - etc.
    """

    # Clean question
    question_clean = question.strip().rstrip("?")

    # Try to extract the meaningful clause after "Which"/"What"
    match = re.search(
        r"(?i)(?:which|what)(?: of (?:the )?(?:following|these))?(.*)", question_clean
    )

    if match:
        # Example: " is a programming language" or " best describes the pool of water"
        context_part = match.group(1).strip()
        if not context_part:
            context_part = question_clean.lower()
    else:
        # Fallback if no pattern matches
        context_part = question_clean.lower()

    # Pick up to 4 random choices
    selected = random.sample(choices, min(4, len(choices)))

    # Build contextualized answers
    # Example: "Python is a programming language" or "Dirty best describes the pool of water"
    labeled_choices = [
        f"{label}. {choice} {context_part}"
        for label, choice in zip("ABCD", selected)
    ]

    # Final formatted output
    formatted = f"{question}\n" + "\n".join(labeled_choices)
    return formatted


# Example 1
q1 = "Which of the following is a programming language?"
choices1 = ["Python", "Ruby", "Car", "Banana", "C++"]
print(make_contextual_choices(q1, choices1))

print("\n")

# Example 2
q2 = "What best describes the pool of water?"
choices2 = ["dirty", "fresh", "boiling", "frozen"]
print(make_contextual_choices(q2, choices2))


Which of the following is a programming language?
A. Ruby is a programming language
B. Python is a programming language
C. Banana is a programming language
D. C++ is a programming language


What best describes the pool of water?
A. boiling best describes the pool of water
B. frozen best describes the pool of water
C. dirty best describes the pool of water
D. fresh best describes the pool of water


In [36]:
print(make_contextual_choices(questions[0], choices[0]))

What is in the motorcyclist's mouth?
A. popsicle stick is in the motorcyclist's mouth
B. food is in the motorcyclist's mouth
C. cigarette is in the motorcyclist's mouth
D. toothpick is in the motorcyclist's mouth


In [38]:
print(make_contextual_choices(questions[1], choices[1]))

Which number birthday is probably being celebrated?
A. ten number birthday is probably being celebrated
B. nine number birthday is probably being celebrated
C. one number birthday is probably being celebrated
D. thirty number birthday is probably being celebrated


In [37]:
print(make_contextual_choices(questions[2], choices[2]))

What best describes the pool of water?
A. boiling best describes the pool of water
B. fresh best describes the pool of water
C. frozen best describes the pool of water
D. dirty best describes the pool of water


In [39]:
print(make_contextual_choices(questions[3], choices[3]))

What is the white substance on top of the cupcakes?
A. icing is the white substance on top of the cupcakes
B. butter is the white substance on top of the cupcakes
C. mayo is the white substance on top of the cupcakes
D. ice cream is the white substance on top of the cupcakes


In [40]:
print(make_contextual_choices(questions[4], choices[4]))

What type of device is sitting next to the laptop?
A. mouse type of device is sitting next to the laptop
B. pen type of device is sitting next to the laptop
C. keyboard type of device is sitting next to the laptop
D. mobile phone type of device is sitting next to the laptop


In [41]:
print(make_contextual_choices(questions[5], choices[5]))

The thing on the animal to the left's head is similar to what is on the head of what else?
A. vampire is on the head of what else
B. zombie is on the head of what else
C. devil is on the head of what else
D. witch is on the head of what else
